In [ ]:
!pip install sentence-transformers
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 29.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.

In [ ]:
# pip install qdrant-client
# pip install sentence-transformers

import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
#QdrantSetup
client = QdrantClient(
    url="URL_DE_SEU_CLUSTER",
    api_key="SUA_API_KEY",
)

In [ ]:
# Carrega o dataset
patent_ds = pd.read_json('Estudo2.json')
patent_ds

,Subclass_labels,Abstract,Title,No
0,[F04C],the invention has a housing a pump section for...,pump device,US08926299
1,[C10L],in a fuel composition containing a static diss...,method of formulating a fuel composition,US08926716
2,"[G02C, C09K, G02F, G02B]",the present invention relates to optical eleme...,optical elements with alignment facilities for...,US08926091
3,[B43M],a device for inserting sheets into an envelope...,device for inserting sheets into an envelope,US08925291


In [ ]:
index_name = "patent-200" # Name of your Cluster

dimensions_768 = 768 #Size of embeddings
model_768 = SentenceTransformer('all-distilroberta-v1') #Loads the pre-trained SBERT model

dimensions_384 = 384 #Size of embeddings
model_384 = SentenceTransformer('all-MiniLM-L6-v2') #Loads the pre-trained SBERT model

dimensions = dimensions_384
model = model_384

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def histogram(classes:list, k:int):
    result = {}
    ctr=1
    for value, key in sorted(((classes.count(e), e) for e in set(classes)), reverse=True):
        if (ctr > k): break
        result[key] = value
        ctr+=1
    return result

In [ ]:
def process_result(accuracy_dict, k, n, type):
    index = "{}-{}-{}".format(k,n,type)
    if (index in accuracy_dict):
        accuracy_dict[index] = accuracy_dict.get(index) + 1
    else:
        accuracy_dict[index] = 1

In [ ]:
def get_process_result(accuracy_dict, k, n, type):
    index = "{}-{}-{}".format(k,n,type)
    if (index in accuracy_dict):
        return accuracy_dict[index]
    else:
        return 0

In [ ]:
def print_process_result(accuracy_dict, k_list, n_list):
    for k in k_list:
        for n in n_list:
            positive = get_process_result(accuracy_dict, k, n, 'positive')
            negative = get_process_result(accuracy_dict, k, n, 'negative')
            accuracy = positive / (positive + negative)
            print("k={} - n={} - Positive: {} - Negative: {} - "
                "Accuracy: {} ".format(k,n,positive,negative,accuracy))

In [ ]:
def transform_process_result(accuracy_dict, k_list, n_list):
    matrix = np.zeros((len(k_list), len(n_list)))
    i = j = 0
    for k in k_list:
        j=0
        for n in n_list:
            positive = get_process_result(accuracy_dict, k, n, 'positive')
            negative = get_process_result(accuracy_dict, k, n, 'negative')
            accuracy = positive / (positive + negative)
            matrix[i][j] = accuracy
            j+=1
        i+=1
    return matrix

In [ ]:
#Method to approximate search
def search(vector):
    hits = client.search(
        collection_name=index_name,
        query_vector=vector,
        limit=100
    )
    return hits

In [ ]:
#Performs queries to group returned patent subclasses for each input patent
print("Index name: ",index_name)
k_list = [1,2,3,4,5,6,7,8,9,10]
n_list = [10,25,50,75,100]
accuracy_dict = {}
ctr_hit = 0
hits = 0
positive = negative = 0
subclass_list = []
hit_list = []
ctr_queries = 0
for index, row in patent_ds.iterrows():
    ctr_queries+=1
    #if (ctr_queries > 2000): break
    embedding = model.encode(row.Title+" "+row.Abstract).tolist()

    hits = search(embedding)

    hit_list.clear()
    hits_count=0
    for hit in hits:
        hit_list.append(hit.payload['subclass_labels'])
        #if (hits_count <= 20): print(str(hit.score) +" - "+hit.payload['subclass_labels'])
        hits_count+=1

    print("Query id: "+str(ctr_queries)+" - Patent No: "+row.No+" - Subclasses: "+
        ';'.join(map(str, row.Subclass_labels))+" - Hits: "+str(hits_count)) #This last part concatenate the subclasses using the ";" character

    k=10
    n=50
    ctr_hit = 0
    for subclass in hit_list:
        ctr_hit+=1
        if (ctr_hit > n): break
        subclass_list.append(subclass)

    histogram_res = histogram(subclass_list, k)
    #print('k: '+str(k)+ ' - n: '+str(n))
    print(histogram_res)
    subclass_list.clear()

    for subclass in row.Subclass_labels:
        #print("Subclass: "+subclass+" Subclass List: "+str(subclass_list))
        if (subclass in histogram_res):
            process_result(accuracy_dict, k, n, 'positive')
        else:
            process_result(accuracy_dict, k, n, 'negative')

Index name:  patent-200
0.73412174 - F04B
0.70929134 - F04C
0.7087494 - F04D
0.7065462 - F04B
0.6823298 - F04F
0.6717811 - F04C
0.6621525 - F04F
0.65507144 - F04C
0.6544111 - F04F
0.6538743 - F04B
0.65322673 - F04D
0.6525836 - F04B
0.6520708 - F04B
0.6486199 - F04B
0.6421088 - F02N
0.6403306 - F04B
0.63852143 - F04B
0.6375731 - F16H
0.6337683 - F04B
0.63334924 - B60W
0.63155055 - B63H
Query id: 1 - Patent No: US08926299 - Subclasses: F04C - Hits: 100
{'F04B': 18, 'F04C': 8, 'F04D': 5, 'F04F': 3, 'F15B': 2, 'A61M': 2, 'G05D': 1, 'F16H': 1, 'F16B': 1, 'F03B': 1}
0.6920314 - C10L
0.6919422 - C10L
0.6876122 - C10L
0.68108565 - C10L
0.6761439 - C10L
0.6643949 - C10L
0.65919167 - C10L
0.6294215 - C10L
0.6263968 - C10L
0.61776125 - C10L
0.61431205 - C10L
0.60935056 - C10L
0.6049459 - C10L
0.60328853 - C10L
0.59944516 - C10L
0.5985603 - C10L
0.59518516 - C10L
0.59508336 - C10L
0.590042 - G01M
0.58848405 - C10L
0.58824664 - C10L
Query id: 2 - Patent No: US08926716 - Subclasses: C10L - Hits: 100